In [44]:
package 'HybridSUV Dynamic Data Roll-Up'{
    import SI::Real;
    import ScalarFunctions::sum;
    value type bps;
    
    package 'System'{
        block 'Body Control Module'  specializes 'Data Roll-Up Pattern'::'Data Roll-up Aspect'{}
        
        block 'Brake Control Module'  specializes 'Data Roll-Up Pattern'::'Data Roll-up Aspect'{}
        
        block 'Braking System' specializes 'Data Roll-Up Pattern'::'Data Roll-up Aspect'{
            part brakeControlModule : 'Brake Control Module' subsets subRollUp;
        }
        
        block 'HybridSUV' specializes 'Data Roll-Up Pattern'::'Data Roll-up Aspect'{
            part powerControlUnit : 'Power Control Unit' subsets subRollUp;
            part brakingSystem : 'Braking System' subsets subRollUp;
        }
        
        block 'Power Control Unit' specializes 'Data Roll-Up Pattern'::'Data Roll-up Aspect'{
            part bodyControlModule : 'Body Control Module' subsets subRollUp;
            part powertrainControlModule : 'Powertrain Control Module' subsets subRollUp;
        }
        
        block 'Powertrain Control Module' specializes 'Data Roll-Up Pattern'::'Data Roll-up Aspect'{}
    } // END package 'System'
    
    package 'Analysis' {
        block 'Test Driver';
        
        block 'Analysis Context'{
            value total : bps;
            
            part hsuv : System::HybridSUV;
            part testDriver : 'Test Driver';
            
            assert constraint equal_rates: EqualRates(
                total_context=total, 
                total_hsuv=hsuv::totalDataRate);
        }
        
        constraint def EqualRates (total_context: Real, total_hsuv: Real) {
            total_context == total_hsuv
        }
    } // END package 'Analysis'
    
    package 'Data Roll-Up Pattern'{
        constraint def TotalChildren (child : Real[*], total : Real, offset : Real) {
            total == sum(child) + offset
        }
        
        
        block 'Data Roll-up Aspect'{
            constraint sum : TotalChildren (
                child = subRollUp::totalDataRate, 
                total = totalDataRate,
                offset = dataRate);
                
            part subRollUp : 'Data Roll-up Aspect'[*];
            
            value 'totalDataRate' : bps;
            value 'dataRate' : bps;
            value 'dataRateInStateOn' : bps;
            value 'dataRateInStateStandby' : bps;          
        } //END block 'Data Roll-up Aspect'
        
        
        value type DStandbySignal;
        value type DTurnOnSignal;
        value type DTurnOffSignal;
        
        
        state def 'Data Roll-up Behavior' {
            entry; then Off;
            
            
            state Off;
            transition off_to_standby
                first Off
                accept DStandbySignal
                then StandBy;
            transition off_to_on
                first Off
                accept DTurnOnSignal
                then On;
            
            state StandBy;
            transition standby_to_off
                first StandBy
                accept DTurnOffSignal
                then Off;
            transition standby_to_on
                first StandBy
                accept DTurnOnSignal
                then On;
            
            state On;
            transition on_to_standby
                first On
                accept DStandbySignal
                then StandBy;
            transition on_to_off
                first On
                accept DTurnOffSignal
                then Off;
        } // END state def 'Data Roll-up Behavior'
    } // END package 'Data Roll-Up Pattern'
    
    
}

Package Dynamic Data Roll-Up (4a6ece3a-ec6d-4506-ba44-4200d0d107ca)
